In [1]:
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
!pip install --disable-pip-version-check torch==1.13.1 torchdata==8.5.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
ERROR: Ignored the following yanked versions: 0.3.0a0
ERROR: Could not find a version that satisfies the requirement torchdata==8.5.1 (from versions: 0.3.0a1, 0.3.0, 0.4.0, 0.4.1, 0.5.0, 0.5.1, 0.6.0, 0.6.1, 0.7.0, 0.7.1)
ERROR: No matching distribution found for torchdata==8.5.1


In [3]:
!pip install \
transformers==4.27.2\
datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 2.7 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig


In [5]:
name='knkarthick/dialogsum'
dataset=load_dataset(name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
example_indices=[40,200]
dash_line='-'.join('' for x in range(100))
for i , index in enumerate(example_indices):
  print(dash_line)
  print('Example : ',i)
  print(dash_line)
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('BaseLine Human Summary')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

---------------------------------------------------------------------------------------------------
Example :  0
---------------------------------------------------------------------------------------------------
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BaseLine Human Summary
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example :  1
-----

In [8]:
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Dialouge Summarization Without Instruction

In [ ]:
sentence='What time it is , Tom ?'
sentence_encoded=tokenizer(sentence,return_tensor='pt')
sentence_decoded=tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True
)
print('ENCODED SENTENCE : ')
print(sentence_encoded['input_ids'][0])
print('\n DECODED SENTENCE : ')
print(sentence_decoded)

In [16]:
for i, index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  input=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example : ',i+1)
  print(dash_line)
  print(f'\033[1;31mINPUT PROMPT :\033[0m \n{dialogue}')
  print(dash_line)
  print(f'\033[1;31mBASELINE HUMAN SUMMARY :\033[0m \n{summary}')
  print(dash_line)
  print(f'\033[1;31mMODEL GENERATION  - WITHOUT PROMPT ENGINEERING :\033[0m \n {output}\n')

---------------------------------------------------------------------------------------------------
Example :  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  - WITHOUT PROMPT ENGINEERING : 
 Person1: It's ten to nine.

---------------------

## Zero shot Learning
### With prompt instruction

In [17]:
for i, index in enumerate(example_indices):
  dialogue= dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
summarise the following conversation.
  {dialogue}
  Summary
  """
  # Input constructed
  input=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example : ',i+1)
  print(dash_line)
  print(f'\033[1;31mINPUT PROMPT : \033[0m\n{dialogue}')
  print(dash_line)
  print(f'\033[1;31mBASELINE HUMAN SUMMARY : \033[0m\n{summary}')
  print(dash_line)
  formatted_output = f"\033[1;31mMODEL GENERATION  -ZERO SHOT PROMPT : \033[0m\n {output}\n"

  # Print the formatted output
  print(formatted_output)

---------------------------------------------------------------------------------------------------
Example :  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  -ZERO SHOT PROMPT : 
 Person1: It's ten to nine.

--------------------------------

In [18]:
for i, index in enumerate(example_indices):
  dialogue= dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Dialouge.
  {dialogue}
What was going on ?
  """
  # Input constructed
  input=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example : ',i+1)
  print(dash_line)
  print(f'\033[1;31mINPUT PROMPT : \033[0m\n{dialogue}')
  print(dash_line)
  print(f'\033[1;31mBASELINE HUMAN SUMMARY : \033[0m\n{summary}')
  print(dash_line)
  formatted_output = f"\033[1;31mMODEL GENERATION  -ZERO SHOT PROMPT : \033[0m\n {output}\n"

  # Print the formatted output
  print(formatted_output)

---------------------------------------------------------------------------------------------------
Example :  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  -ZERO SHOT PROMPT : 
 Person1: It's ten to nine.

--------------------------------

## One Shot Learning

In [19]:
def make_prompt(example_indices_full,example_indices_to_summarise):
  prompt=''
  for  index in example_indices_full:
    dialogue= dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    # the stop sequence '{summary}\n\n is important for FLAN-15
    prompt+=f"""
  Dialouge
  {dialogue}
  What was going on ?
  {summary}

  """
  dialogue = dataset['test'][example_indices_to_summarise]['dialogue']
  prompt+=f"""
    Dialouge
  {dialogue}
  What was going on ?
  """
  return prompt


In [20]:
example_indices_full=[40]
example_index_to_summarise=200
one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarise)
print(one_shot_prompt)



  Dialouge
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there. 
  What was going on ?
  #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

  
    Dialouge
  #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And

In [22]:
summary=dataset['test'][example_index_to_summarise]['summary']

# Input constructed
input=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

print(dash_line)
print(f'\033[1;31mBASELINE HUMAN SUMMARY : \033[0m\n{summary}')
print(dash_line)
formatted_output = f"\033[1;31mMODEL GENERATION  -ONE SHOT PROMPT : \033[0m\n {output}\n"

# Print the formatted output
print(formatted_output)

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  -ONE SHOT PROMPT : 
 #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software.



##Few shot Inference

In [23]:
example_indices_full=[40,80,128]
example_index_to_summarise=200
few_shot_prompt=make_prompt(example_indices_full,example_index_to_summarise)
print(few_shot_prompt)


  Dialouge
  #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there. 
  What was going on ?
  #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

  
  Dialouge
  #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#:

In [24]:
summary=dataset['test'][example_index_to_summarise]['summary']

# Input constructed
input=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )

print(dash_line)
print(f'\033[1;31mBASELINE HUMAN SUMMARY : \033[0m\n{summary}')
print(dash_line)
formatted_output = f"\033[1;31mMODEL GENERATION  -FEW SHOT PROMPT : \033[0m\n {output}\n"

# Print the formatted output
print(formatted_output)

Token indices sequence length is longer than the specified maximum sequence length for this model (828 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  -FEW SHOT PROMPT : 
 #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.



## Generation Configuration

In [27]:
generation_config=GenerationConfig(max_new_tokes=50)
summary=dataset['test'][example_index_to_summarise]['summary']

# Input constructed
input=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          generation_config=generation_config,
      )[0],
      skip_special_tokens=True
  )

print(dash_line)
print(f'\033[1;31mBASELINE HUMAN SUMMARY : \033[0m\n{summary}')
print(dash_line)
formatted_output = f"\033[1;31mMODEL GENERATION  -FEW SHOT PROMPT : \033[0m\n {output}\n"

# Print the formatted output
print(formatted_output)


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY : 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION  -FEW SHOT PROMPT : 
 #Person1 wants to upgrade his computer. #Person2 wants to add a

